In [2]:
import torch
!pip install datasets
!pip install transformers

In [3]:
from transformers import Trainer, TrainingArguments
from transformers import BertForSequenceClassification, BertTokenizer
model_name = 'bert-base-multilingual-cased'
model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = BertTokenizer.from_pretrained(model_name)
# model.train()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [4]:
from datasets import load_dataset
path_to_load = '/content/en-train.json'
train_dataset, val_dataset = load_dataset('json', data_files=path_to_load, split=['train[:80%]','train[80%:]'])
train_dataset = train_dataset.rename_column('text','sentence')
val_dataset = val_dataset.rename_column('text','sentence')
train_dataset.features

from pprint import pprint
pprint(train_dataset.info.__dict__)

print()
#print('Max sentence length: ', max([len(sen) for sen in train_dataset['sentence'] + test_dataset['sentence']]))
print('Max sentence length: ', max([len(sen) for sen in train_dataset['sentence'] + val_dataset['sentence']]))

Using custom data configuration default-4c0460a974b3bf39


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-4c0460a974b3bf39/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.
{'builder_name': 'json',
 'citation': '',
 'config_name': 'default',
 'dataset_size': 10180325,
 'description': '',
 'download_checksums': {'/content/en-train.json': {'checksum': '85ac02e6d5df94ccdbcfc47a987087ca21c646fcce9915d0d2e519d42952f72e',
                                                   'num_bytes': 10366199}},
 'download_size': 10366199,
 'features': {'id': Value(dtype='int64', id=None),
              'label': Value(dtype='int64', id=None),
              'sentence': Value(dtype='string', id=None)},
 'homepage': '',
 'license': '',
 'post_processed': None,
 'post_processing_size': None,
 'size_in_bytes': 20546524,
 'splits': {'train': SplitInfo(name='train', num_bytes=10180325, num_examples=9324, dataset_name='json')},
 'supervised_keys': None,
 'version': 0.0.0}



In [8]:
# Set voacb size
sentence_lenght = 200

# Check embeddings
batch_sentences = train_dataset['sentence'][:3]
print(batch_sentences)
encoded = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors='pt', max_length=sentence_lenght)
print(encoded)

print()
for ids in encoded["input_ids"]:
    pprint(tokenizer.decode(ids))


# Build embeddings on vocab
def tokenize(batch):
    return tokenizer(batch['sentence'], padding=True, truncation=True, max_length=sentence_lenght)  # batch['text']


train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
#test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))

train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
#test_dataset.set_format('torch', columns=['input_ids', 'attention_mask'])
val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

["Schools 'need to adopt green approach'\nPUBLISHED : Monday, 08 March, 1999, 12:00am\nYou are never too young to learn to look after the environment, an official says.\nSpeaking at a seminar on environmental studies held at the Open University of Hong Kong, Mabel Mak Lok-sheung, senior environmental protection officer at the Environmental Protection Department, said concern for the environment should be taught from an early age.\nSchools should adopt a green approach to help create a better living environment, Ms Mak said.\nEnvironmental awareness would penetrate every level of society if children learned it at school, she said.\n'It should be a top-down approach. The first level is the management level.\n'Principals and administrative staff at schools should set an example for teachers ", 'IN SCHOOL\nKerber, defending Graf’s record, gets the message\nJanuary 29, 2016 00:00 IST\nAngelique Kerber\nI think she [Serena Williams] ', 'Microsoft launches Xbox Music on Web for free\nPUBLISHE

In [9]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


training_args = TrainingArguments(
    run_name='try',     # used for `wandb <https://www.wandb.com/>`_ logging.
    output_dir='./results',
    num_train_epochs=4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    learning_rate=2e-5,  #2e-5
    weight_decay=0.01,
    evaluation_strategy="epoch",
    logging_dir='./logs',
    # deepspeed='./ds_config.json'   # with cpu offload crashes, without it is slow in GoogleColab
    # fp16=True                      # without it is faster,
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

In [10]:
trainer.train()
trainer.evaluate()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: ignored

In [ ]:
# %load_ext tensorboard
# %tensorboard --logdir logs